In [ ]:
import yaml
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go

import glog as log

import evaluation.tools as evt
from evaluation.evaluation_lib import aggregate_all_results

In [ ]:
# Parse experiment yaml file
experiments_path="../experiments/full_euroc.yaml"

# Get experiment information from yaml file.
experiment_params = yaml.load(open(experiments_path))

# Get directory where all results are stored
results_dir = os.path.expandvars(experiment_params['results_dir'])

# Collect results
stats = aggregate_all_results(results_dir)
#evt.check_stats(stats)

In [ ]:
figure = evt.draw_ape_boxplots_plotly(stats, False)
figure.show()